# Modeling a Pincell
In this module, we'll demonstrate the basic features of the Python API for constructing input files and running OpenMC. In it, we will show how to create a basic reflective pincell model that is equivalent to modeling an infinite array of fuel pins in a pressurized water reactor. We highly recommend having a copy of the [Python API reference documentation](https://docs.openmc.org/en/stable/pythonapi/index.html) open in another browser tab that you can refer to.

<img src="pincell.png" alt="drawing" width="250"/>

For this example, we'll create a simple pincell that is composed of:
- UO<sub>2</sub> with 3.5 weight% enriched in U-235 at 11 g/cm<sup>3</sup>
- zirconium clad at 6.5 g/cm<sup>3</sup>
- H<sub>2</sub>O moderator at 1.0 g/cm<sup>3</sup>

The dimensions of our fuel pin will be as follows:
- Fuel outer radius = 0.46955 cm
- Clad inner radius = 0.47910 cm
- Clad outer radius = 0.54640 cm
- Fuel pin pitch = 1.44270 cm

In [ ]:
import openmc

## Basics of Jupyter Notebook

You are working within a Jupyter notebook. Some common commands which will be useful:
- To execute a cell: `Shift+Enter`
- To insert a cell above or below: `Esc+a`, `Esc+b`

## Naming Conventions

Before we start working with OpenMC's Python API, it's helpful to understand the naming convention of objects so that you can conceptualize what is a function, what is a class, etc. OpenMC's Python interface follows the same naming convention that is adopted by many/most Python projects:

- Module names are `lowercase`
- Classes are `CamelCase`
- Functions and class attributes/variables are `lowercase_with_underscores`

To give a few specific examples:

- `openmc.deplete` is the depletion _module_
- `openmc.run` is a _function_
- `openmc.Material` is a _class_
- `openmc.StatePoint` is a _class_

## Setting Attributes

When building OpenMC models, we will work with many different classes. Each class typically has _attributes_, a variable belonging to the class. When creating a class, you can often set those attributes directly when you instantiate the object, like this:

In [ ]:
my_cell = openmc.Cell(name="box")

Or, you can assign values to attributes after you have already created the object. For example, the following is equivalent to the above:

In [ ]:
my_other_cell = openmc.Cell()
my_other_cell.name = "box2"

You can generally also mix-and-match, setting some attributes when you instantiate the object, and others at a later point.

In [ ]:
new_cell = openmc.Cell(name="box3")
new_cell.temperature = 500


## How to Get Help/Learn More

When building OpenMC models, we recommend having a copy of the Python API documentation (linked earlier) open. You can also query information about classes and methods directly from Jupyter. For example, we can use `help` to get documentation on all of the valid attributes for functions and classes.

In [ ]:
help(openmc.Cell)

Attributes which have a default value will appear as `<attribute>=<the default value>` in the class/function definition. For example, in the above we see that the default name for a cell is the empty string, `''`.

For classes, all of the member functions you can access will then be defined further down, with their associated documentation. To see documentation for just one specific function, you can use syntax like:

In [ ]:
my_cell = openmc.Cell()
help(my_cell.clone)

You can also query the type of a particular object using `type`. Note that we can also display the attributes for a particular object using `print`.

In [ ]:
type(my_cell)

In [ ]:
print(my_cell)

In order to quickly see all of the member functions on a class, press `Tab` to perform tab-completion to view a drop-down list on all the options available to you. You can then do `Shift+Tab` to see the documentation for each function inline.

In [ ]:
#my_cell.

## OpenMC Model

The OpenMC `Model` class houses all of the pieces of a Monte Carlo simulation. We will assemble our pincell by progressively adding to a model, and then run that model.

In [ ]:
model = openmc.Model()

### The `cross_sections.xml` file

The `cross_sections.xml` tells OpenMC where it can find nuclide cross sections and $S(\alpha,\beta)$ tables. It serves the same purpose as MCNP's `xsdir` file and Serpent's `xsdata` file. As we mentioned, this can be set either by the `OPENMC_CROSS_SECTIONS` environment variable or the `Materials.cross_sections` attribute.

Let's have a look at what's inside this file:

In [ ]:
!cat $OPENMC_CROSS_SECTIONS | head -n 10
print('    ...')
!cat $OPENMC_CROSS_SECTIONS | tail -n 10

## Defining Materials

Materials in OpenMC are defined as a set of nuclides with specified atom/weight fractions. To begin, we will create a material by making an instance of the `Material` class. In OpenMC, many objects, including materials, are identified by a "unique ID" (a positive integer). You can also give a material a `name` as well.

In [ ]:
zirconium = openmc.Material(1, "zirconium")
print(zirconium)

One great feature in OpenMC's design is that the user interaction is designed to be as streamlined as possible. You don't *need* to set either an ID or a name (because often you can simply refer to an object with the actual object handle). If you were to create a material without any ID, OpenMC will assign a default for you.

In [ ]:
mat = openmc.Material()
print(mat)

We see that an ID of 2 was automatically assigned. Let's now move on to adding nuclides to our material. The `Material` object has a method `add_element()` whose first argument is the name of the nuclide and second argument is the atom or weight fraction. We see that by default it assumes we want an atom fraction.

In [ ]:
#zirconium.add_element

In [ ]:
zirconium.add_element('Zr', 1.0)
zirconium.set_density('g/cm3', 6.5)

In [ ]:
print(zirconium)

### Element Expansion
We can see that OpenMC automatically inserted the natural abundances of the zirconium isotopes for us! How convenient! The way this feature works is as follows:

- First, it checks whether `Materials.cross_sections` has been set, indicating the path to a `cross_sections.xml` file.
- If `Materials.cross_sections` isn't set, it looks for the `OPENMC_CROSS_SECTIONS` environment variable.
- If either of these are found, it scans the file to see what nuclides are actually available and will expand elements accordingly.

Let's build our fuel material. For sake of illustration, let's suppose that we had O-16, but not natural oxygen in our fuel. Also note that OpenMC has a convenient feature to set the nuclide concentrations for weight percent enrichments in U-235.

In [ ]:
# Add nuclides to uo2
uo2 = openmc.Material(name="uo2")
uo2.add_element('U', 1.0, enrichment=3.5)
uo2.add_nuclide('O16', 2.0)
uo2.set_density('g/cm3', 10.0)

With UO2 finished, let's now create materials for the coolant.

In [ ]:
water = openmc.Material(name="water")
water.add_element('H', 2.0)
water.add_nuclide('O16', 1.0)
water.set_density('g/cm3', 1.0)

An astute observer might now point out that this water material we just created will only use free-atom cross sections. We need to tell it to use an $S(\alpha,\beta)$ table so that the bound atom cross section is used at thermal energies. To do this, there's an `add_s_alpha_beta()` method.

In [ ]:
water.add_s_alpha_beta('c_H_in_H2O')

In [ ]:
!ls $OPENMC_CROSS_SECTIONS
!ls /Users/anovak/projects/cross_sections/endfb-vii.1-hdf5/
!ls /Users/anovak/projects/cross_sections/endfb-vii.1-hdf5/neutron

We are done with our materials -- now we just need to register them in our model.

In [ ]:
model.materials = openmc.Materials([uo2, zirconium, water])
print(model.materials)

### Material Modifications

The power of OpenMC's Python API is the ability to easily modify and control your model.

In [ ]:
print(water)

water.remove_nuclide('O16')

print(water)

water.add_element('O', 1.0)

print(water)

We see that now O16 and O17 were automatically added. O18 is missing because our cross sections file (which is based on ENDF/B-VII.1) doesn't have O18.

### Python Lists

Many of OpenMC's classes are derived from Python's `list` type, and can therefore be appended, popped, etc.

In [ ]:
print(isinstance(model.materials, list))

In [ ]:
new_fuel = openmc.Material(name="new_fuel")
new_fuel.add_nuclide('Pu239', 1.0)
new_fuel.add_nuclide('Si28', 2.0)
new_fuel.set_density('g/cm3', 9.0)

In [ ]:
model.materials.append(new_fuel)
print(model.materials)

## Defining Geometry

We now need to define the geometry. One way to do so is to use constructive solid geometry (CSG), also known as combinatorial geometry. The object that allows us to assign a material to a region of space is called a `Cell` (same concept in MCNP, for those familiar). There are four stages in building a cell:

#### Surfaces
In order to define a region that we can assign to a cell, we must first define surfaces which bound the region. A *surface* is a locus of zeros of a function of Cartesian coordinates $x$, $y$, and $z$, e.g.

- A plane perpendicular to the x axis: $x - x_0 = 0$
- A cylinder parallel to the z axis: $(x - x_0)^2 + (y - y_0)^2 - R^2 = 0$
- A sphere: $(x - x_0)^2 + (y - y_0)^2 + (z - z_0)^2 - R^2 = 0$

The full [list of available surfaces](https://docs.openmc.org/en/stable/pythonapi/base.html#building-geometry) is as follows.

Planes:

- `openmc.Plane` — An arbitrary plane of the form $Ax + By + Cz = D$
- `openmc.XPlane` — A plane perpendicular to the x axis of the form $x - x_0 = 0$
- `openmc.YPlane` — A plane perpendicular to the y axis of the form $y - y_0 = 0$
- `openmc.ZPlane` — A plane perpendicular to the z axis of the form $z - z_0 = 0$

Quadrics:

- `openmc.XCylinder` — An infinite cylinder whose length is parallel to the x-axis of the form $(y - y_0)^2 + (z - z_0)^2 = r^2$
- `openmc.YCylinder` — An infinite cylinder whose length is parallel to the x-axis of the form $(x - x_0)^2 + (z - z_0)^2 = r^2$
- `openmc.ZCylinder` — An infinite cylinder whose length is parallel to the x-axis of the form $(x - x_0)^2 + (y - y_0)^2 = r^2$
- `openmc.Sphere` — A sphere of the form $(x - x_0)^2 + (y - y_0)^2 + (z - z_0)^2 = r^2$
- `openmc.XCone` — A cone parallel to the x-axis of the form $(y - y_0)^2 + (z - z_0)^2 = r^2 (x - x_0)^2$
- `openmc.YCone` — A cone parallel to the y-axis of the form $(x - x_0)^2 + (z - z_0)^2 = r^2 (y - y_0)^2$
- `openmc.ZCone` — A cone parallel to the z-axis of the form $(x - x_0)^2 + (y - y_0)^2 = r^2 (z - z_0)^2$
- `openmc.Quadric` — A generic quadric surface

Torii:

- `openmc.XTorus` — A torus of the form $(x - x_0)^2/B^2 + (\sqrt{(y - y_0)^2 + (z - z_0)^2} - A)^2/C^2 - 1 = 0$
- `openmc.YTorus` — A torus of the form $(y - y_0)^2/B^2 + (\sqrt{(x - x_0)^2 + (z - z_0)^2} - A)^2/C^2 - 1 = 0$
- `openmc.ZTorus` — A torus of the form $(z - z_0)^2/B^2 + (\sqrt{(x - x_0)^2 + (y - y_0)^2} - A)^2/C^2 - 1 = 0$

#### Half-Spaces

<img src="csg_half.png" alt="drawing" width="600"/>

A surface *half-space* is the region whose points satisfy a positive or negative inequality of the surface equation. For example, for a sphere of radius one centered at the origin, the surface equation is $f(x,y,z) = x^2 + y^2 + z^2 - 1 = 0$. Thus, we say that the negative half-space of the sphere is defined as the collection of points satisfying $f(x,y,z) < 0$, which one can reason is the inside of the sphere. Conversely, the positive half-space of the sphere would correspond to all points outside of the sphere.

#### Regions
A region is then a combination of (or just one) half-spaces.

#### Fills
Finally, a cell is complete once we have defined what is _filling_ the cell, which may be one of:

- material
- nothing (`None`), or vacuum/void
- universe
- lattice

In [ ]:
sphere = openmc.Sphere(r=1.0)

Note that by default the sphere is centered at the origin so we didn't have to supply `x0`, `y0`, or `z0` arguments. Strictly speaking, we could have omitted `r` as well since it defaults to one. To get the negative or positive half-space, we simply need to apply the `-` or `+` unary operators, respectively.

In [ ]:
inside_sphere = -sphere
outside_sphere = +sphere
type(inside_sphere)

Now let's see if `inside_sphere` actually contains points inside the sphere:

In [ ]:
print((0,0,0) in inside_sphere, (0,0,2) in inside_sphere)
print((0,0,0) in outside_sphere, (0,0,2) in outside_sphere)

Everything works as expected! Now that we understand how to create half-spaces, we can create more complex volumes by combining half-spaces using Boolean operators: `&` (intersection), `|` (union), and `~` (complement):

- `&`: logical AND
- `|`: logical OR
- `~`: logical NOT

For example, let's say we want to define a region that is the top part of the sphere (all points inside the sphere that have $z > 0$.

In [ ]:
z_plane = openmc.ZPlane(0)
northern_hemisphere = -sphere & +z_plane

In [ ]:
cell = openmc.Cell()
cell.region = northern_hemisphere

# or...
cell = openmc.Cell(region=northern_hemisphere)

cell.fill = water

### Universes and in-line plotting

A collection of cells is known as a universe and can be used as a repeatable unit when creating a model. Although we don't need it yet, the benefit of creating a universe is that we can visualize our geometry while we're creating it.

In [ ]:
universe = openmc.Universe()
universe.add_cell(cell)

# this also works
universe = openmc.Universe(cells=[cell])

The `Universe` object has a `plot` method that will display our the universe as current constructed:

In [ ]:
universe.plot(width=(2.0, 2.0))

By default, the plot will appear in the $x$-$y$ plane. We can change that with the `basis` argument.

In [ ]:
universe.plot(width=(2.0, 2.0), basis='xz')

If we have particular fondness for, say, fuchsia, we can tell the `plot()` method to make our cell that color.

In [ ]:
universe.plot(width=(2.0, 2.0), basis='xz',
              colors={cell: 'fuchsia'})

### Boundary Conditions

To specify boundary conditions, you simply need to set the `Surface.boundary_type` to one of:

- `transmission` (default)
- `vacuum`
- `reflective`
- `periodic` (either rotational or translational)
- `white` (isotropic angular flux)

<img src="mc_bcs.png" alt="drawing" width="600"/>

In [ ]:
northern_hemisphere.boundary_type = 'vacuum'

### Pin cell geometry

We now have enough knowledge to create our pin-cell. We need three surfaces to define the fuel and clad:

1. The outer surface of the fuel -- a cylinder parallel to the z axis
2. The inner surface of the clad -- same as above
3. The outer surface of the clad -- same as above

These three surfaces will all be instances of `openmc.ZCylinder`, each with a different radius according to the specification.

In [ ]:
fuel_outer_radius = openmc.ZCylinder(r=0.46955)
clad_inner_radius = openmc.ZCylinder(r=0.47910)
clad_outer_radius = openmc.ZCylinder(r=0.54640)

With the surfaces created, we can now take advantage of the built-in operators on surfaces to create regions for the fuel, the gap, and the clad:

In [ ]:
fuel_region = -fuel_outer_radius
gap_region = +fuel_outer_radius & -clad_inner_radius
clad_region = +clad_inner_radius & -clad_outer_radius

We will also create two z-planes in order to bound the geometry in the axial direction.

In [ ]:
top = openmc.ZPlane(z0=150.0, boundary_type='vacuum')
bot = openmc.ZPlane(z0=-150.0, boundary_type='vacuum')
layer = +bot & -top

Now we can create corresponding cells that assign materials to these regions. As with materials, cells have unique IDs that are assigned either manually or automatically. Note that the gap cell doesn't have any material assigned (it is void, a.k.a. vacuum, by default).

In [ ]:
fuel = openmc.Cell()
fuel.fill = uo2
fuel.region = fuel_region & layer

gap = openmc.Cell()
gap.region = gap_region & layer

clad = openmc.Cell()
clad.fill = zirconium
clad.region = clad_region & layer

Finally, we need to handle the coolant outside of our fuel pin. To do this, we create x- and y-planes that bound the geometry.

In [ ]:
pitch = 1.44270
left = openmc.XPlane(-pitch/2, boundary_type='reflective')
right = openmc.XPlane(pitch/2, boundary_type='reflective')
front = openmc.YPlane(-pitch/2, boundary_type='reflective')
back = openmc.YPlane(pitch/2, boundary_type='reflective')

The water region is going to be everything outside of the clad outer radius and within the box formed as the intersection of four half-spaces.

In [ ]:
water_region = +left & -right & +front & -back & +clad_outer_radius & layer

moderator = openmc.Cell()
moderator.fill = water
moderator.region = water_region

The final step is to assign the cells we created to a universe and tell OpenMC that this universe is the "root" universe in our geometry.

In [ ]:
root_universe = openmc.Universe(cells=(fuel, gap, clad, moderator))
model.geometry = openmc.Geometry(root_universe)

In [ ]:
root_universe.plot((0, 0, 0), width=(pitch, pitch), pixels=100000)

## Starting source and settings

The Python API has a module `openmc.stats` with various univariate and multivariate probability distributions. We can use these distributions to create a starting source using the `openmc.Source` object. One can independently specify the spatial distribution (`space`), the angular distribution (`angle`), the energy distribution (`energy`), and the time distribution (`time`). For this example, we'll only specify the spatial distribution as a single point.

In [ ]:
# Create a point source
point = openmc.stats.Point((0, 0, 0))
source = openmc.IndependentSource(space=point)

In [ ]:
model.settings.source = source
model.settings.batches = 100
model.settings.inactive = 10
model.settings.particles = 1000

## Running OpenMC

Running OpenMC from Python can be done using the `model.run()` function. This function allows you to set the number of MPI processes and OpenMP threads, if need be.

In [ ]:
statepoint = model.run()

## Geometry plotting

We saw before that we could call the `Universe.plot()` method to show a universe while we were creating our geometry. There is also a built-in plotter in the codebase that is much faster than the Python plotter and has more options. The interface looks somewhat similar to the `Universe.plot()` method. Instead though, we create `Plot` instances, assign them to a `Plots` collection, export it to XML, and then run OpenMC in geometry plotting mode. As an example, let's specify that we want the plot to be colored by material (rather than by cell) and we assign yellow to fuel and blue to water.

In [ ]:
plot = openmc.Plot()
plot.filename = 'pinplot'
plot.width = (pitch, pitch)
plot.pixels = (200, 200)
plot.color_by = 'material'
plot.colors = {uo2: 'yellow', water: 'blue'}

With our plot created, we need to add it to a `Plots` collection which can be exported to XML.

In [ ]:
model.plots = openmc.Plots([plot])

Now we can run OpenMC in plotting mode by calling the `plot_geometry()` function.

In [ ]:
model.plot_geometry()

Now, we can use functionality from IPython to display the `.png` image inline in our notebook:

In [ ]:
!ls

from IPython.display import Image
Image("pinplot.png")

## Tallies

In this section, we'll be looking at how to extract custom information from an OpenMC simulation in what is known as a "tally." A tally accumulates statistical information during the simulation about particles when they eneter regions of phase space specified on the tally. The limits of these regions are set by "filters" applied to the tally. Scores and nuclides can also be applied to tallies to indicate what type of information is kept about the particle (e.g. reaction types, flux, heat, etc.).

Any tally in OpenMC can be described with the following form:

$$ 
 X = \underbrace{\int d\mathbf{r} \int d\mathbf{\Omega} \int
    dE}_{\text{filters}} \underbrace{f(\mathbf{r}, \mathbf{\Omega},
    E)}_{\text{scores}} \underbrace{\psi (\mathbf{r}, \mathbf{\Omega}, E)}_{\text{angular flux}}
$$

where filters set the limits of the integrals and the scoring function is convolved with particle information (e.g. reaction type, current material, etc.). For example, if you wanted to calculate the fission reaction rate caused by fast neutrons in cell 3, your tally becomes

$$ 
 X = \int_\text{cell 3} d\mathbf{r} \int_{4\pi} d\mathbf{\Omega} \int_{1 MeV}^{20 MeV}
    dE \ \ \Sigma_f(\mathbf{r}, \mathbf{\Omega},
    E) \psi (\mathbf{r}, \mathbf{\Omega}, E)
$$

<div class="alert alert-block alert-info">
A full list of scores and their meanings can be found <a href=https://docs.openmc.org/en/stable/usersguide/tallies.html#scores >here</a>.
</div>

In this exercise we'll be adding tallies to perform a few different tasks:


  **1. Determine the energy and heat produced per fission** \
  **2. Plot the flux spectrum of the pincell** \
  **3. Plot reaction types based on material**
  
First, to determine the recoverable energy produced per fission we'll create a tally without filters to gather information on the fission reaction rate ("`fission`") and recoverable fission energy ("`kappa-fission`"). Because we want this information talllied throughout the model, a "global" tally, no filters need to be applied.

### Task 1: Energy released per fission

In [ ]:
tally1 = openmc.Tally()
tally1.scores = ['fission', 'kappa-fission']
print(tally1)

In [ ]:
model.tallies = openmc.Tallies([tally1])

Now, we can simply re-run our model with this additional tally.

In [ ]:
statepoint = model.run()

If we list our current directory, we see that several new files have been created as a result of this run: `summary.h5`, `tallies.out`, and `statepoint.50.h5`. The `tallies.out` file contains a text output of all user-specified tallies for the simulation. The summary file contains information about the simulation's setup (geometry, materials, meshes, etc.) in an HDF5 format.

In [ ]:
!cat tallies.out

This can be useful to quickly look at simple tally results, but isn't a great format to post-process simulation data. For that we'll look to the statepoint file. The statepoint file contains information about simulation results including tally specifications and data.

In [ ]:
print(statepoint)

To extract information from the statepoint file we'll create an `openmc.StatePoint` object. The `statepoint.get_tally` function will search for tallies by scores, filters, nuclides, ids, and return the closest match. Exact matches can be speficied as well.

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    my_tally = sp.get_tally(scores=['fission', 'kappa-fission'])
    print(my_tally)
    
    my_tally = sp.get_tally(id=tally1.id)
    print(my_tally)

<div class="alert alert-block alert-info">
<b>A quick aside on how statepoint objects interact with summary files:</b>


The `openmc.statepoint` object will read information from the `summary.h5` file if one is present, keeping that file open in the Python interpreter. The open `summary.h5` file can interfere with the initialization of subsequent OpenMC simulations. It is recommended that information be extracted from statepoints within a [context manager](https://book.pythontips.com/en/latest/context_managers.html) as we do here. Alternatively, making sure to call the `openmc.StatePoint.close` method will work also. For more details please look to the [relevant section in the user's guide](https://docs.openmc.org/en/stable/usersguide/troubleshoot.html#runtimeerror-failed-to-open-hdf5-file-with-mode-w-summary-h5).   
</div>

To compute the energy released per fission event, we can simply take the tallied energy released per fission and divide it by the fission rate.

In [ ]:
#my_tally.get_values

In [ ]:
fission_rate = my_tally.get_values(scores=['fission']).flatten()[0]
kappa_fission_rate = my_tally.get_values(scores=['kappa-fission']).flatten()[0]
ev_per_fission = kappa_fission_rate / fission_rate
mev_per_fission = ev_per_fission * 1e-6
print('MeV per fission: ', mev_per_fission)

In [ ]:
fission_rate_std_dev = my_tally.get_values(scores=['fission'], value='std_dev').flatten()[0]
kappa_fission_rate_std_dev = my_tally.get_values(scores=['kappa-fission'],value='std_dev').flatten()[0]

from uncertainties import ufloat
f = ufloat(fission_rate, fission_rate_std_dev) 
kf = ufloat(kappa_fission_rate, kappa_fission_rate_std_dev) 

print('MeV per fission: {:.6f}'.format(kf / f * 1e-6))

As with most values coming out of an MC code, these values are per source-particle. In this case these units cancel out, but this will not be the case in our next example.

### Task 2: Plot the neutron flux spectrum

To perform this task, we'll be applying a tally with an energy filter and a score. OpenMC's data module contains different group structures. For this problem we'll use the CASMO-70 group structure. An energy filter can easily be created from a pre-defined group structure in OpenMC as follows:

In [ ]:
print(openmc.mgxs.GROUP_STRUCTURES.keys())

In [ ]:
energy_filter = openmc.EnergyFilter.from_group_structure('CASMO-70')
len(energy_filter.bins)

In [ ]:
spectrum_tally = openmc.Tally()
spectrum_tally.filters = [energy_filter]
spectrum_tally.scores = ['flux']

In [ ]:
model.tallies += [spectrum_tally]
statepoint = model.run()

In [ ]:
!cat tallies.out

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    tally = sp.get_tally(id=spectrum_tally.id)
    tally = sp.get_tally(scores=['flux'])
    tally = sp.get_tally(filters=spectrum_tally.filters)

In [ ]:
values = tally.get_values(scores=['flux']).flatten()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

bin_boundaries = energy_filter.lethargy_bin_width

#print(energy_filter.bins)
plt.step(np.unique(energy_filter.bins)[:-1], values / bin_boundaries)

plt.xscale('log')
plt.xlabel(r'Energy (eV)')
plt.ylabel(r'Flux ($\frac{particle-cm}{source-particle}$)')
plt.show()

### Normalizing Tallies

Note that the units of flux in the above plot are in $\frac{particle-cm}{source-particle}$. As is the case with many values tallied by Monte Carlo codes, the value of the flux does not account for volume and is in terms of the number of source particles emitted. To generate this same plot in terms of absolute flux units ($\frac{particle}{cm^{2}-s}$) we'll need to normalize this tally by:

  - the volume of the region the tally covers
  - the number of source particle emitted

In this case, the volume of the region is the volume of the entire pincell, which we can obtain using a bounding box (or the dimensions we set earlier, but let's introduce the notion of a bounding box!)

In [ ]:
print(root_universe.bounding_box)
ur = root_universe.bounding_box.upper_right
ll = root_universe.bounding_box.lower_left
volume = np.prod(ur-ll)

Determining the number of source particles per second is more complicated, however. This means computing the eV/source particle due to fission. To get the source rate, we'll need the following pieces of information:

  1. the total power produced in the tally region (known a priori)
  2. the heat produced by fission power, per source particle
  
 To get this information we'll need to construct another tally to get additional information from the simulation.

In [ ]:
pincell_power = 200 # in Watts
source_tally = openmc.Tally()
source_tally.scores = ['kappa-fission']
model.tallies += [source_tally]

In [ ]:
statepoint = model.run()

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    out = sp.get_tally(id=source_tally.id)
    heating = out.get_values(scores=['kappa-fission']).flatten()[0]

The combination of the following tally values and power provide us with the source normalization needed as follows:


$$ \text{neutron source} [\frac{n}{s}] = \text{power} [\frac{J}{s}] \times \frac{1}{1.6\times 10^{-19}} [\frac{eV}{J}] \times \frac{1}{\text{heat per fission} [\frac{eV}{source}]} $$ 

In [ ]:
J_to_eV = 1 / 1.6e-19
neutron_source = pincell_power * J_to_eV * (1 / heating)
print('Neutron source (n/s): ', neutron_source)

In [ ]:
normalized_spectrum = neutron_source * values / volume

In [ ]:
plt.step(np.unique(energy_filter.bins)[:-1], normalized_spectrum / bin_boundaries)
plt.xscale('log')
plt.xlabel(r'Energy (eV)')
plt.ylabel(r'Flux ($\frac{particle}{cm^{-2} s}$)')
plt.show()

### Task 3: Reaction Types by Material

Looking at the different reaction types by material will require a material filter and the set of reaction types we want to score. For this example, we'll be scoring absorption, scattering and fission in each material.

To start, we'll create a material filter.

In [ ]:
print(model.materials)

In [ ]:
material_filter = openmc.MaterialFilter(model.materials)
material_tally = openmc.Tally()
material_tally.filters = [material_filter]
material_tally.scores = ['absorption', 'scatter', 'fission']

In [ ]:
model.tallies += [material_tally]

In [ ]:
statepoint = model.run()

Now we'll gather information from the statepoint file about each score we applied to the tally. With multiple scores and materials, we'll use a Pandas data frame to view the results in a more coherent manner.

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    tally = sp.get_tally(filters=[material_filter])
    absorption = tally.get_values(scores=['absorption']).flatten()
    scatter = tally.get_values(scores=['scatter']).flatten()
    fission = tally.get_values(scores=['fission']).flatten()
    df = tally.get_pandas_dataframe()

In [ ]:
df

In [ ]:
df['normalized-mean (rxn/s)'] = neutron_source * df['mean']
df

We'll add a new entry in the dataframe for our material names to make plotting easier.

In [ ]:
# get all materials from the geometry
materials = model.geometry.get_all_materials()

# set names based on matching material IDs
for mat_id, material in materials.items():
    df.loc[df['material'] == mat_id, 'mat_name'] = material.name
df

### Task 4: Mesh Tallies

In order to restrict a tally to a particular region in space, OpenMC supports (i) cell, (ii) structured mesh, and (iii) unstructured mesh tallies. For unstructured mesh tallies, you need to compile OpenMC with libMesh enabled. We don't have this dependency set up on the Collab instance, so we'll just work with structured mesh tallies.

In [ ]:
mesh = openmc.RegularMesh()
mesh.lower_left = (-pitch/2, -pitch/2)
mesh.upper_right = (pitch/2, pitch/2)
mesh.dimension = (50, 50)

mesh_filter = openmc.MeshFilter(mesh)
heat = openmc.Tally()
heat.scores = ['kappa-fission']
heat.filters = [mesh_filter]
model.tallies += [heat]

In [ ]:
statepoint = model.run()

In [ ]:
with openmc.StatePoint(statepoint) as sp:
    mesh_tally_out = sp.get_tally(id=heat.id)
    
mesh_flux = mesh_tally_out.get_values()
mesh_flux = mesh_flux.reshape(mesh.dimension)

In [ ]:
plt.figure()
img = plt.imshow(mesh_flux)
plt.colorbar(img)

In [ ]:
mesh_flux = mesh_tally_out.get_values(value='std_dev')
mesh_flux = mesh_flux.reshape(mesh.dimension)
plt.figure()
img = plt.imshow(mesh_flux)
plt.colorbar(img)

## Universes

A universe is a collection of cells that can be used as a repeatable unit in the geometry. At a minimum, there must be one "root" universe (say, named `root`), which gets passed to `openmc.Geometry(root)`. But you can also use universes to repeat a collection of cells multiple times throughout a geometry. Here, we will explore some basic features of universes.

We'll start by making a universe which looks similar to the pincell we built earlier - say, a cylinder of UO2 enclosed by an infinite region of water. First, we set up our materials and create our geometry.

In [ ]:
print(model.materials)

In [ ]:
pin_surface = openmc.ZCylinder(r=1.0)
inside_pin = -pin_surface

pin_cell = openmc.Cell(region=inside_pin, fill=uo2)
outside_cell = openmc.Cell(region=~inside_pin, fill=water)

In [ ]:
universe = openmc.Universe()
universe.add_cells([pin_cell, outside_cell])

We have created a universe containing a pin, enclosed in an infinite medium of water. Now let's suppose that I want to fill this universe into an enclosing cell, a cylinder of radius 5 cm. Let's first create this cylinder, and then we will fill it with our `universe`.

In [ ]:
big_cylinder = openmc.ZCylinder(r=5.0)
big_cell = openmc.Cell()
big_cell.region = -big_cylinder
big_cell.fill = universe

Let's take a look at our geometry. In order to visualize at this stage, we need to create a universe from our `big_cell`.

In [ ]:
big_universe = openmc.Universe(cells=[big_cell])
big_universe.plot(width=(10.0, 10.0))

We can see that our `big_cell`, the large cylinder, has been filled with the `universe` we declared earlier. Let's increase the complexity a bit to understand how this filling works. What if the cylinder of UO2 in our `universe` is not located at the origin, but is instead shifted to a different position?

In [ ]:
pin_surface.x0 = 1.0
pin_surface.y0 = 1.5
big_universe.plot(width=(10.0, 10.0))

We see that when we fill a universe inside of another cell, that there's (by default) no transformation of coordinates. You can shift the position of the universe filling a cell with the `Cell.translation` attribute. There are similar adjustments you can make, like rotations.

In [ ]:
big_cell.translation = [-1.0, 0.0, 0.0]
big_universe.plot(width=(10.0, 10.0))

## Lattices

Lattices are a convenient way to (i) repeat a universe multiple times in space, while (ii) automatically translating that universe's origin to different positions in space. In order to explore this concept, let's work with a PWR fuel assembly.

In [ ]:
colors = {
    water: 'lightblue',
    zirconium: 'gray',
    uo2: 'red'
}

We will make individual universes for each repeatable unit in the assembly design.

### Fuel Pin

In [ ]:
pitch = 1.26
fuel_or = openmc.ZCylinder(r=0.39)
clad_ir = openmc.ZCylinder(r=0.40)
clad_or = openmc.ZCylinder(r=0.46)

fuel = openmc.Cell(fill=uo2, region=-fuel_or)
gap = openmc.Cell(region=+fuel_or & -clad_ir)
clad = openmc.Cell(fill=zirconium, region=+clad_ir & -clad_or)
moderator = openmc.Cell(fill=water, region=+clad_or)

fuel_pin = openmc.Universe(cells=[fuel, gap, clad, moderator])
fuel_pin.plot(width=(pitch, pitch), color_by='material', colors=colors)

### Guide Tube

In [ ]:
guide_clad_ir = openmc.ZCylinder(r=0.56)
guide_clad_or = openmc.ZCylinder(r=0.60)

guide_inner = openmc.Cell(fill=water, region=-guide_clad_ir)
guide_clad = openmc.Cell(fill=zirconium, region=+guide_clad_ir & -guide_clad_or)
guide_outer = openmc.Cell(fill=water, region=+guide_clad_or)

guide_tube = openmc.Universe(cells=[guide_inner, guide_clad, guide_outer])
guide_tube.plot(width=(pitch, pitch), color_by='material', colors=colors)

Now we are ready to build our fuel assembly, by stamping these two universes into a repeatable pattern. OpenMC has `RectLattice` and `HexLattice` objects, to create lattices on a Cartesian grid or on a hexagonal grid. For our fuel assembly, we need to use `RectLattice`. 

When creating a rectangular lattice, we need to define:

1. The lower-left coordinates of the lattice (`.lower_left`)
2. The size of each lattice element (`.pitch`)
3. The 2D arrangement of universes (`.universes`)
4. (_optionally_) A universe that is used outside of the defined region (`.outer`)

In [ ]:
lattice = openmc.RectLattice()

# Define the lower-left coordinates and size of lattice elements
lattice.lower_left = (-pitch, -pitch)
lattice.pitch = (pitch, pitch)

# Now we specify what is actually inside the lattice.  This 2D lattice will be a
# list of lists like
# lattice.universes = [ [a1, a2, ...], [b1, b2, ...], ...]
# The inner lists specify columns from left to right.  The outer lists specify
# rows from top to bottom.
lattice.universes = [
    [fuel_pin,   fuel_pin],
    [guide_tube, fuel_pin]
]

# We also should specify what is outside of the lattice.  In this case, it is
# the infinite water universe.
all_water = openmc.Universe(cells=[openmc.Cell(fill=water)])
lattice.outer = all_water

To plot the lattice, we need to put it in a universe. For this, we'll create a single cell filled with the lattice, and then put that single cell inside a universe to plot:

In [ ]:
u = openmc.Universe(cells=[openmc.Cell(fill=lattice)])
u.plot(width=(3*pitch, 3*pitch), color_by='material', colors=colors)

### What exactly does `outer` mean?

In the previous section, we set the lattice outer universe to a universe containing a single cell with only water in it. To get a better sense of what the outer universe does, let's change the outer universe to the guide tube universe:

In [ ]:
lattice.outer = guide_tube
u.plot(width=(3*pitch, 3*pitch), color_by='material', colors=colors)

### 17x17 Fuel Assembly

In [ ]:
guide_tube_positions = [
    (2, 5), (2, 8), (2, 11),
    (5, 2), (5, 5), (5, 8), (5, 11), (5, 14),
    (8, 2), (8, 5), (8, 8), (8, 11), (8, 14),
    (11, 2), (11, 5), (11, 8), (11, 11), (11, 14),
    (14, 5), (14, 8), (14, 11)
]

In [ ]:
lattice = openmc.RectLattice()

lattice.pitch = (pitch, pitch)
lattice.outer = all_water

# I want (x0, y0) = (0, 0) to be the center of the instrument tube so that means
# the lower-left will be -half a pin pitch in x and y.
assembly_pitch = 17*pitch
lattice.lower_left = (-assembly_pitch/2, -assembly_pitch/2)

# Most of the lattice positions are fuel pins so rather than type all of those
# out, we use a list comprehension to start with a 17x17 array of fuel.
lattice.universes = [[fuel_pin] * 17] * 17

for row, col in guide_tube_positions:
    lattice.universes[row, col] = guide_tube

Now we just have to add the boundary conditions and root universe to finish the geometry. To create a box containing the lattice, we'll use the `rectangular_prism` function:

In [ ]:
box = openmc.model.rectangular_prism(assembly_pitch, assembly_pitch, boundary_type='reflective')
main_cell = openmc.Cell(fill=lattice, region=box)

In [ ]:
u = openmc.Universe(cells=(main_cell,))
u.plot(width=(assembly_pitch, assembly_pitch), color_by='material', colors=colors, pixels=(500,500))

## TRISO Particles

OpenMC includes a few convenience functions for generating locations of randomly packed spheres that can be used to model TRISO particles and/or pebbles in a reactor core. To be clear, this capability is not a stochastic geometry capability like that included in MCNP. It's also important to note that OpenMC does not use delta tracking, which would normally speed up calculations in geometries with tons of surfaces and cells. However, the computational burden can be eased by placing random spheres in a lattice.

This capability relies on three functions/classes:
- `openmc.model.pack_spheres` -- generate locations of random spheres
- `openmc.model.TRISO` -- Cell-like object that holds a universe storing the internal structure of a pebble/TRISO
- `openmc.model.create_triso_lattice` -- Creates a lattice containing `TRISO` objects for improved tracking performance

Let's start with the `pack_spheres` function. This function takes an outer radius of the spheres, a containing region, and a packing fraction and will return an array of sphere coordinates. For our example, let's use spheres with a radius of 1 cm and a packing fraction of 30%. We'll put our spheres inside of a finite cylinder.

In [ ]:
#openmc.model.pack_spheres

In [ ]:
r_cylinder = 10.
h_cylinder = 20.
cylinder = openmc.model.RightCircularCylinder(
    center_base=(0., 0., -10.), 
    height=h_cylinder,
    radius=r_cylinder
)

r_sphere = 1.0
centers = openmc.model.pack_spheres(radius=r_sphere, region=-cylinder, pf=0.3)
centers.shape

In [ ]:
centers[:10]

Now we need to actually creates cells for each of these spheres. To do so, we'll use the `TRISO` class. We'll need to define a universe that we want to fill each sphere. We'll create a universe with a single, infinite cell:

In [ ]:
cell = openmc.Cell(fill=uo2)
sphere_univ = openmc.Universe(cells=[cell])

Now we can create a `TRISO` object for each sphere center, in order to create one cell for each TRISO region.

In [ ]:
trisos = [openmc.model.TRISO(outer_radius=r_sphere, fill=sphere_univ, center=center) for center in centers]
trisos[0]

Let's confirm that the packing fraction of our TRISOs is actually about 30%.

In [ ]:
import math
volume_trisos = len(trisos)*4/3*math.pi*r_sphere**3
volume_cyl = math.pi * r_cylinder**2 * h_cylinder
volume_trisos / volume_cyl

In [ ]:
region_background = openmc.Intersection([~t.region for t in trisos])
background_cell = openmc.Cell(region=region_background & -cylinder)
univ = openmc.Universe(cells=[background_cell] + trisos)
univ.plot(width=(20., 20.))

While this works in principle, it will lead to **very** poor tracking performance; every time a particle reaches the background cell, it has to determine the distance to the boundary of _every single_ sphere. To improve tracking performance, we can use the `create_triso_lattice` function to overlay a lattice that limits how many distance checks need to be performed.

In [ ]:
lower_left, upper_right = (-cylinder).bounding_box
shape = (5, 5, 5)
pitch = (upper_right - lower_left)/shape
openmc.model.create_triso_lattice
lattice = openmc.model.create_triso_lattice(
    trisos=trisos,
    lower_left=lower_left,
    pitch=pitch,
    shape=shape,
    background=zirconium
)

In [ ]:
c = openmc.Cell(fill=lattice, region=-cylinder)
univ = openmc.Universe(cells=[c])
univ.plot(width=(20., 20.))

In [ ]:
univ.plot(width=(20., 20.), color_by='material')